In [1]:
!pip uninstall scipy -y
!pip install scipy==1.10.0
!pip install Gpy GPyOpt

Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 437.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.0 MB/s eta 0:00:00
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83602 sha256=3c33316b4f1e5f66ea9382507efb5598fe1f25dffb707274eaf7df89f8be4846
  Stored in directory: /root/.cache/pip/wheels/72/17/52/9d818b4c60f733bf49d5cf82bc2758ebbdc57a0471137c37be
Successfully built GPyOpt


In [2]:
!pip install tensorflow==2.9.0 tensorflow-probability==0.16.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully u

In [ ]:
import os
os.kill(os.getpid(), 9)


In [1]:
from google.colab import drive

import pandas as pd
from sklearn.linear_model import BayesianRidge
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error, accuracy_score
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import GPy
import GPyOpt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib
import pickle

Import models

In [2]:
drive.mount('/content/drive')

date_model = tf.keras.models.load_model('/content/drive/MyDrive/my_bnn_model_savedmodel')


# Load the scaler object from Google Drive
scaler = joblib.load('/content/drive/MyDrive/scaler_amt.pkl')

with open('/content/drive/MyDrive/gpr_model.pkl', 'rb') as file:
    amt_model = pickle.load(file)


label_path = '/content/drive/My Drive/le_merchant.pkl'

# Load the LabelEncoder
with open(label_path, 'rb') as file:
    le_merchant = pickle.load(file)
mer_model_path = '/content/drive/My Drive/best_model.pkl'

# Load the model from the specified path
mer_model = joblib.load(mer_model_path)

# Define the path to the file in Google Drive
file_path = '/content/drive/My Drive/kbins_discretizer.pkl'

# Load the fitted KBinsDiscretizer
kb_loaded = joblib.load(file_path)


Mounted at /content/drive


Data Transform

In [3]:

def data_transform(data):
  data['P ORIGINAL GROSS AMT BIN'] = kb_loaded.transform(data[['ORIGINAL GROSS AMT']])


  data = data.rename(columns={"ORIGINAL GROSS AMT": "P ORIGINAL GROSS AMT","MERCHANT NAME":"P MERCHANT NAME"})
  #le=LabelEncoder()

  data['P MERCHANT NAME'] = le_merchant.fit_transform(data['P MERCHANT NAME'])

  data['TRANS DATE'] = pd.to_datetime(data['TRANS DATE'], errors='coerce')

  data['P_YEAR'] = data['TRANS DATE'].dt.year
  data['P_MONTH'] = data['TRANS DATE'].dt.month
  data['P_DAY'] = data['TRANS DATE'].dt.day
  data['P_DAY_OF_WEEK'] = data['TRANS DATE'].dt.dayofweek

  data_date=data#[['P_YEAR','P_MONTH','P_DAY','P_DAY_OF_WEEK','CARD NUMBER','P MERCHANT NAME','P ORIGINAL GROSS AMT' ]]
  data_amt=data[['P_YEAR', 'P_MONTH', 'P_DAY', 'P_DAY_OF_WEEK', 'P MERCHANT NAME','P ORIGINAL GROSS AMT', 'CARD NUMBER']]#
  data_mer=data[['CARD NUMBER',	'P ORIGINAL GROSS AMT BIN'	,'P_YEAR'	,'P_MONTH'	,'P_DAY','P MERCHANT NAME']]
  return data_date,data_amt, data_mer
#data_date,data_amt, data_mer=data_transform(data)

DATE MODEL

In [4]:


def date_pred(data):
     data_date=data[['P_YEAR','P_MONTH','P_DAY','P_DAY_OF_WEEK','CARD NUMBER','P MERCHANT NAME','P ORIGINAL GROSS AMT' ]]
     data_scaled = tf.convert_to_tensor(data_date, dtype=tf.float32)
     y=date_model.predict(data_scaled)
     y = np.argmax(y, axis=1)
    # Add days from 'days_to_add' to the 'dates'
     new = data['TRANS DATE'] + pd.to_timedelta(y, unit='days')

     return new
#date_pred(data_date)

Amount model

In [5]:

def amt_predict(data):
# Use the scaler to transform new data
  data_scaled = scaler.transform(data)
  data_scaled

  #scaler = StandardScaler()
  #data_scaled = scaler.fit_transform(data_date)
  y,var=amt_model.predict(data_scaled)
  y=y.astype(float)
  return y
#amt_predict(data_amt)

Merchant Model

In [6]:
import numpy as np

# Assuming le_merchant was used to fit on the training data
def find_closest_label(unseen_label, seen_labels):
    # Find the closest label in the seen labels
    closest_label = min(seen_labels, key=lambda x: abs(x - unseen_label))
    return closest_label
mer=[0,2,1,5,9,10,20,30,40,60,90,1000,2000,3000,40000]
def label_decode(mer):

      try:
          demer = le_merchant.inverse_transform(mer)
      except ValueError as e:
          print("Error:", e)

          # Get the classes seen by the encoder
          seen_labels = set(le_merchant.transform(le_merchant.classes_))

          # Initialize an array to store decoded merchants
          demer = []

          for label in mer:
              if label in seen_labels:
                  demer.append(le_merchant.inverse_transform([label])[0])
              else:
                  # Find the closest label for the unseen label
                  closest_label = find_closest_label(label, seen_labels)
                  demer.append(le_merchant.inverse_transform([closest_label])[0])

      # Convert demer to numpy array or keep as a list
      demer = np.array(demer)
      return demer
#label_decode(mer)

In [7]:
def mer_predict(data):
    decode_mer = []

    # Loop through each row in the data
    for i in range(len(data)):
        try:
            # Predict for the specific row of data
            row_data = data.iloc[[i]]  # Select row as DataFrame
            mer = mer_model.predict(row_data)  # Predict for the specific row

            # Try decoding the predicted value
            try:
                transformed_value = label_decode(mer)[0]  # Assuming label_decode returns an array
                decode_mer.append(transformed_value)
            except :
                # If there's an error in decoding, append 'others'
                decode_mer.append('others')

        except :
            # If there's an error in predicting for that row, append 'others'
            decode_mer.append('others')

    return decode_mer





Predict transaction at once

In [8]:

def all_predictions(data,n):
        updated_transactions=None
        data_sorted = data.sort_values(by=['CARD NUMBER', 'TRANS DATE'], ascending=[True, False])
        updated_transactions = data_sorted.groupby('CARD NUMBER').first().reset_index()
        original_index = updated_transactions.index
        for n in range(0,n):
            updated_transactions['TRANS DATE'] = pd.to_datetime(updated_transactions['TRANS DATE'])

            latest_transactions = updated_transactions.loc[updated_transactions.groupby('CARD NUMBER')['TRANS DATE'].idxmax()].reset_index(drop=True)
            #latest_transactions = updated_transactions.groupby('CARD NUMBER').first().reset_index()

            x_date, x_amt, x_mer = data_transform(latest_transactions)
            # Generate predictions
            date = date_pred(x_date)
            amt = amt_predict(x_amt)
            #mer = mer_predict(x_mer)
            #if x_mer.shape[1] != mer_model.n_features_in_:
            #   print("Error: Number of features in x_mer does not match mer_model.")
            # Handle the error, e.g., by resizing x_mer or retraining mer_model
            #continue
            mer = mer_predict(x_mer)

            amt = amt.ravel()
            # Create a new DataFrame with the predicted values
            new_transaction = pd.DataFrame({
                'CARD NUMBER': latest_transactions['CARD NUMBER'],  # Or any specific card number logic
                'TRANS DATE': date,
                'MERCHANT NAME': mer,
                'ORIGINAL GROSS AMT': amt})

            updated_transactions = pd.concat([updated_transactions, new_transaction], ignore_index=True)
            #updated_transactions.append(new_transaction)
        updated_transactions = updated_transactions.drop(index=original_index)
        updated_transactions['ORIGINAL GROSS AMT']=updated_transactions['ORIGINAL GROSS AMT'].astype(float)
        updated_transactions['ORIGINAL GROSS AMT']=updated_transactions['ORIGINAL GROSS AMT'].round(2)
        return updated_transactions
#all_predictions(df,1)

Prediction of single card number

In [15]:

def card_predictions(data,n, card):
        updated_transactions=None
        updated_transactions = data.loc[data['CARD NUMBER'] ==card].sort_values('TRANS DATE', ascending=False).head(1).reset_index()

        original_index = updated_transactions.index
        for n in range(0,n):
            new_transaction = None
            updated_transactions['TRANS DATE'] = pd.to_datetime(updated_transactions['TRANS DATE'])

            latest_transactions = updated_transactions.loc[updated_transactions['TRANS DATE'] == updated_transactions['TRANS DATE'].max()].head(1).reset_index(drop=True)
            #latest_transactions = updated_transactions.groupby('CARD NUMBER').first().reset_index()
            #print('lll',latest_transactions)
            #print(updated_transactions)
            x_date, x_amt, x_mer = data_transform(latest_transactions)
            # Generate predictions
            date = date_pred(x_date)
            amt = amt_predict(x_amt)
            #mer = mer_predict(x_mer)
            #if x_mer.shape[1] != mer_model.n_features_in_:
            #   print("Error: Number of features in x_mer does not match mer_model.")
            # Handle the error, e.g., by resizing x_mer or retraining mer_model
            #continue
            mer = mer_predict(x_mer)

            amt = amt.ravel()
            # Create a new DataFrame with the predicted values
            new_transaction = pd.DataFrame({
                'CARD NUMBER': latest_transactions['CARD NUMBER'],  # Or any specific card number logic
                'TRANS DATE': date,
                'MERCHANT NAME': mer,
                'ORIGINAL GROSS AMT': amt})

            updated_transactions = pd.concat([updated_transactions, new_transaction], ignore_index=True)
            #updated_transactions.append(new_transaction)
        updated_transactions = updated_transactions.drop(index=original_index)

        return updated_transactions
#card_predictions(df,1,8577)

# FRONT END

In [10]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://bvc2nnslsiw-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [16]:
from flask import Flask, request, render_template_string, send_file
import pandas as pd
import io

app = Flask(__name__)

# Global variables
uploaded_file = None
processed_data = None  # Initialize processed_data as None

# HTML template with upload form, number input boxes, and calculate button
upload_form = """
<!doctype html>
<html lang="en">
  <head>
    <title>CONSUMER SPENDING PREDICTOR</title>
  </head>
  <style>
      body {
        display: flex;
        flex-direction: column;
        align-items: flex-start;   /* Align items to the left */
        padding-left: 50px;    /* Add 50px padding on the left */
        height: 100vh;
        margin: 20px;
      }
      form {
        text-align: left; /* Align form elements to the left */
      }
      input[type="submit"] {
      margin-top: 10px; /* less space */
    }
    </style>
  <body>
    <h1>Consumer Spending Predictor</h1>
    <form method="POST" enctype="multipart/form-data">
      <div>
        <style>
  .input-container {
    display: block; /* Ensures each input container takes up the full width */
    margin-bottom: 10px; /* Add spacing between input boxes */
  }

  .input-container label {
    display: block; /* Ensures the label takes up the full width */
    margin-bottom: 5px; /* Add spacing between label and input */
  }

  .input-container input[type="number"] {
    width: 200px; /* Make input boxes take up full width of their container */
  }

</style>

<div class="input-container">
  <label for="number1">Enter Number of Predictions:</label>
  <input type="number" id="number1" name="number1" required>
</div>

<div class="input-container">
  <label for="number2">Enter Card Number:</label>
  <input type="number" id="number2" name="number2">
</div>
      <div>
        <label for="file">Upload a file:</label>
        <input type="file" name="file">
      </div>
      <div>
      <label for="wholeFilePrediction">Future Transactions for Uploaded File:
      <input type="submit" id="wholeFilePrediction" name="action" value="Predict for Whole File">

      </div>
      <div>
      <label for="cardPrediction">Future Transactions for Card Number:
      <input type="submit" id="cardPrediction" name="action" value="Predict for Card Number">

      </div>

    </form>
    {% if message %}
    <p style="color: green;">{{ message }}</p>
    {% endif %}
    {% if result is not none %}
    {% endif %}
    {% if table %}
    <h2>{{ button_name }} Results</h2>  <!-- Display the button name as the header -->
    {{ table|safe }}
    <form action="/download" method="GET">
      <button type="submit">Save CSV</button> <!-- Save button appears after table is displayed -->
    </form>
    {% endif %}
  </body>
</html>
"""

# Dummy preprocess function to load data into a DataFrame
def preprocess_data(file_stream):
    try:
        file_stream.seek(0)  # Go to the beginning of the stream
        df = pd.read_csv(file_stream)
    except Exception as e:
        df = pd.DataFrame()  # If there's an issue, return an empty DataFrame
        print(f"Error processing file: {e}")
    return df

@app.route("/", methods=["GET", "POST"])
def home():
    global uploaded_file, processed_data
    message = ""
    table = None
    button_name = None

    if request.method == "POST":
        number1 = request.form.get("number1")
        number2 = request.form.get("number2")
        action = request.form.get("action")
        file = request.files.get("file")

        if file and file.filename != '':
            uploaded_file = io.BytesIO(file.read())
            message = "File uploaded successfully!"
            print(f"File uploaded and stored in memory.")

        if uploaded_file:
            data = preprocess_data(uploaded_file)
            if not data.empty:
                table = data.to_html()
            else:
                message = "The uploaded file appears to be empty or invalid."

        if action == "Predict for Whole File":
            try:
                num1 = int(number1)
                if uploaded_file:
                    df = preprocess_data(uploaded_file)
                    if not df.empty:
                        data = all_predictions(df, num1)
                        table = data[['TRANS DATE','CARD NUMBER','MERCHANT NAME','ORIGINAL GROSS AMT']].to_html()
                        message = "Prediction for whole file completed."
                        processed_data = data
                        button_name = "Prediction for Whole File"
                    else:
                        message = "Please upload a valid file before calculating."
                else:
                    message = "Please upload a file before calculating."
            except ValueError:
                message = "Invalid input. Please enter valid numbers."

        elif action == "Predict for Card Number":
            try:
                num1 = int(number1)
                num2 = int(number2)
                if uploaded_file:
                    df = preprocess_data(uploaded_file)
                    if not df.empty:
                        data = card_predictions(df, num1, num2)
                        table = data[['TRANS DATE','CARD NUMBER','MERCHANT NAME','ORIGINAL GROSS AMT']].to_html()
                        processed_data = data
                        message = "Prediction for card number completed."
                        button_name = "Prediction for Card Number"
            except ValueError:
                message = "Invalid input. Please enter valid numbers."

    return render_template_string(upload_form, message=message, table=table, button_name=button_name)

@app.route("/download", methods=["GET"])
def download():
    global processed_data
    if processed_data is not None:
        output = io.StringIO()
        processed_data.to_csv(output, index=False)
        output.seek(0)

        return send_file(
            io.BytesIO(output.getvalue().encode()),
            mimetype="text/csv",
            as_attachment=True,
            download_name="predictions.csv"
        )
    else:
        return "No data available to download."

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Sep/2024 15:55:33] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Sep/2024 15:55:33] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


File uploaded and stored in memory.
1/1 [==============================] - 0s 25ms/step
Error: y contains previously unseen labels: [1667]
1/1 [==============================] - 0s 36ms/step
Error: y contains previously unseen labels: [1667]
1/1 [==============================] - 0s 35ms/step
Error: y contains previously unseen labels: [1667]
1/1 [==============================] - 0s 54ms/step
Error: y contains previously unseen labels: [1667]
1/1 [==============================] - 0s 24ms/step


INFO:werkzeug:127.0.0.1 - - [06/Sep/2024 15:55:48] "POST /?authuser=0 HTTP/1.1" 200 -


Error: y contains previously unseen labels: [1667]


INFO:werkzeug:127.0.0.1 - - [06/Sep/2024 15:55:49] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


In [12]:
def mer_predict(data):
    decode_mer = []

    # Loop through each row in the data
    for i in range(len(data)):
        try:
            # Predict for the specific row of data
            row_data = data.iloc[[i]]  # Select row as DataFrame
            mer = mer_model.predict(row_data)  # Predict for the specific row

            # Try decoding the predicted value
            try:
                transformed_value = label_decode(mer)[0]  # Assuming label_decode returns an array
                decode_mer.append(transformed_value)
            except :
                # If there's an error in decoding, append 'others'
                decode_mer.append('others')

        except :
            # If there's an error in predicting for that row, append 'others'
            decode_mer.append('others')

    return decode_mer

# Load and transform the data
df = pd.read_csv('/content/drive/MyDrive/data.csv')
df = df.head(3)
#x, y, df = data_transform(df)

# Predict and handle potential errors
mer = all_predictions(df,5)
print(mer)


1/1 [==============================] - 0s 28ms/step
Error: y contains previously unseen labels: [1077]
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [64]
1/1 [==============================] - 0s 34ms/step
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [73]
1/1 [==============================] - 0s 33ms/step
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [73]
1/1 [==============================] - 0s 34ms/step
Error: y contains previously unseen labels: [12]
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [1236]
1/1 [==============================] - 0s 31ms/step
Error: y contains previously unseen labels: [1477]
Error: y contains previously unseen labels: [1477]
    CARD NUMBER TRANS DATE MERCHAN